In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for nes-py: filename=nes_py-8.1.8-cp38-cp38-win_amd64.whl size=47359 sha256=2c6b3593645118204cf3541df22b0b1fb909bc62a6b0225be49bccf0ab89117b
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\8d\6e\f0\113c979eba40def28ee9b3c81a4adec00386106d81fb3bc2c2
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701358 sha256=05b69e2e296718dd98f2a4b0a3a63132fe323ddb27a9926300d75b0983a5da3c
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\78\28\77\b0c74e80a2a4faae0161d5c53bc4f8e436e77aedc79136ee13
Successfully built nes-py gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-met

In [2]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.1
    Uninstalling torchvision-0.10.1:
      Successfully uninstalled torchvision-0.10.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.9.1
    Uninstalling torchaudio-0.9.1:
      Successfully uninstalled torchaudio-0.9.1


In [3]:
!pip install stable-baselines3[extra]

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616826 sha256=446a82ad274f5e76f594500136fe8028fb770d33db0eae7bf05bf612c282353c
  Stored in directory: c:\users\computing\appdata\local\pip\cache\wheels\27\6d\b3\a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=446447 sha256=4bfaaf257d33a0604

In [4]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3.common.callbacks import BaseCallback

In [5]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [6]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [7]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [8]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [9]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [10]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [17]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [18]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,seed=42,n_steps=512) 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=2000000, callback=callback)

Logging to ./logs/PPO_8
----------------------------
| time/              |     |
|    fps             | 276 |
|    iterations      | 1   |
|    time_elapsed    | 1   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 853          |
|    ep_rew_mean          | 1.09e+03     |
| time/                   |              |
|    fps                  | 165          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0032224907 |
|    clip_fraction        | 0.0205       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.663       |
|    explained_variance   | 0.915        |
|    learning_rate        | 1e-06        |
|    loss                 | 20.7         |
|    n_updates            | 9620         |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.05e+03    |
|    ep_rew_mean          | 1.54e+03    |
| time/                   |             |
|    fps                  | 123         |
|    iterations           | 11          |
|    time_elapsed         | 45          |
|    total_timesteps      | 5632        |
| train/                  |             |
|    approx_kl            | 0.003082077 |
|    clip_fraction        | 0.00859     |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.629      |
|    explained_variance   | 0.799       |
|    learning_rate        | 1e-06       |
|    loss                 | 67.3        |
|    n_updates            | 9710        |
|    policy_gradient_loss | -0.00318    |
|    value_loss           | 165         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.22

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.3e+03      |
|    ep_rew_mean          | 1.87e+03     |
| time/                   |              |
|    fps                  | 119          |
|    iterations           | 21           |
|    time_elapsed         | 90           |
|    total_timesteps      | 10752        |
| train/                  |              |
|    approx_kl            | 0.0016807863 |
|    clip_fraction        | 0.000586     |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.463       |
|    explained_variance   | 0.728        |
|    learning_rate        | 1e-06        |
|    loss                 | 72.7         |
|    n_updates            | 9810         |
|    policy_gradient_loss | -0.000476    |
|    value_loss           | 209          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.39e+03     |
|    ep_rew_mean          | 1.88e+03     |
| time/                   |              |
|    fps                  | 118          |
|    iterations           | 31           |
|    time_elapsed         | 133          |
|    total_timesteps      | 15872        |
| train/                  |              |
|    approx_kl            | 0.0012947852 |
|    clip_fraction        | 0.0148       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.562       |
|    explained_variance   | 0.851        |
|    learning_rate        | 1e-06        |
|    loss                 | 268          |
|    n_updates            | 9910         |
|    policy_gradient_loss | -0.00144     |
|    value_loss           | 308          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.45e+03      |
|    ep_rew_mean          | 1.96e+03      |
| time/                   |               |
|    fps                  | 117           |
|    iterations           | 40            |
|    time_elapsed         | 173           |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 0.00040742802 |
|    clip_fraction        | 0.00586       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.328        |
|    explained_variance   | 0.846         |
|    learning_rate        | 1e-06         |
|    loss                 | 167           |
|    n_updates            | 10000         |
|    policy_gradient_loss | 0.000873      |
|    value_loss           | 298           |
-------------------------------------------
-----------------------------------------
| rollout/                |       

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.59e+03     |
|    ep_rew_mean          | 2.08e+03     |
| time/                   |              |
|    fps                  | 116          |
|    iterations           | 416          |
|    time_elapsed         | 1827         |
|    total_timesteps      | 212992       |
| train/                  |              |
|    approx_kl            | 0.0012475683 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.731       |
|    explained_variance   | 0.905        |
|    learning_rate        | 1e-06        |
|    loss                 | 74.1         |
|    n_updates            | 13760        |
|    policy_gradient_loss | -0.00218     |
|    value_loss           | 178          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.62e+03      |
|    ep_rew_mean          | 2.11e+03      |
| time/                   |               |
|    fps                  | 116           |
|    iterations           | 426           |
|    time_elapsed         | 1872          |
|    total_timesteps      | 218112        |
| train/                  |               |
|    approx_kl            | 0.00021470769 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.638        |
|    explained_variance   | 0.802         |
|    learning_rate        | 1e-06         |
|    loss                 | 63.1          |
|    n_updates            | 13860         |
|    policy_gradient_loss | 2.63e-05      |
|    value_loss           | 174           |
-------------------------------------------
-----------------------------------------
| rollout/                |       

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.61e+03     |
|    ep_rew_mean          | 2.08e+03     |
| time/                   |              |
|    fps                  | 116          |
|    iterations           | 436          |
|    time_elapsed         | 1915         |
|    total_timesteps      | 223232       |
| train/                  |              |
|    approx_kl            | 0.0014900196 |
|    clip_fraction        | 0.00879      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.355       |
|    explained_variance   | 0.855        |
|    learning_rate        | 1e-06        |
|    loss                 | 129          |
|    n_updates            | 13960        |
|    policy_gradient_loss | 0.00107      |
|    value_loss           | 227          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.62e+03     |
|    ep_rew_mean          | 2.09e+03     |
| time/                   |              |
|    fps                  | 116          |
|    iterations           | 446          |
|    time_elapsed         | 1959         |
|    total_timesteps      | 228352       |
| train/                  |              |
|    approx_kl            | 0.0024112929 |
|    clip_fraction        | 0.0332       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.552       |
|    explained_variance   | 0.885        |
|    learning_rate        | 1e-06        |
|    loss                 | 42.5         |
|    n_updates            | 14060        |
|    policy_gradient_loss | -0.00106     |
|    value_loss           | 99.2         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.63e+03     |
|    ep_rew_mean          | 2.11e+03     |
| time/                   |              |
|    fps                  | 116          |
|    iterations           | 456          |
|    time_elapsed         | 2004         |
|    total_timesteps      | 233472       |
| train/                  |              |
|    approx_kl            | 0.0031570715 |
|    clip_fraction        | 0.00332      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.752       |
|    explained_variance   | 0.111        |
|    learning_rate        | 1e-06        |
|    loss                 | 671          |
|    n_updates            | 14160        |
|    policy_gradient_loss | -0.00333     |
|    value_loss           | 1.63e+03     |
------------------------------------------
